In [ ]:
import json

# 打开并读取JSON文件
with open('dureadqg/train.json', 'r') as file:
    data = json.load(file)

# 输出读取到的数据
print(data)

In [17]:
import json

# 定义一个空列表用于存储所有的字典
train_list = []
test_list = []
# 打开JSON Lines文件
with open('dureadqg/train.json', 'r') as file:
    # 逐行读取
    for line in file:
        # 解析每一行
        data_dict = json.loads(line)
        # 将解析后的字典添加到列表中
        train_list.append(data_dict)
# 打开JSON Lines文件
with open('dureadqg/dev.json', 'r') as file:
    # 逐行读取
    for line in file:
        # 解析每一行
        data_dict = json.loads(line)
        # 将解析后的字典添加到列表中
        test_list.append(data_dict)


In [18]:
print(len(train_list), len(test_list))

14520 984


In [ ]:
!pip install datasets transformers rouge-score nltk

In [19]:
from datasets import *
train_dataset = Dataset.from_list(train_list)
test_dataset = Dataset.from_list(test_list)
print(train_dataset, test_dataset)

Dataset({
    features: ['context', 'answer', 'question', 'id'],
    num_rows: 14520
}) Dataset({
    features: ['context', 'answer', 'question', 'id'],
    num_rows: 984
})
